# Import Libraries

In [16]:
import numpy as np
from scipy.optimize import linprog
import cvxpy as cp


# Problem 1

In [7]:
c = np.array([-2,-6,-3,2,6,3])
G = np.array([[0,2,1,0,-2,-1],[8,2,3,-8,-2,-3],[2,1,1,-2,-1,-1]])
h = np.array([4,7,2])
A = np.array([[1,4,5,-1,-4,-5],[3,2,9,-3,-2,-9]])
b = np.array([0,5])

In [9]:
re = linprog(c=c,
       A_eq=A,
       b_eq=b,
       A_ub=G,
       b_ub=h)

In [12]:
re.x[:3] - re.x[3:]

array([ 0.90526315, -0.75263158,  0.42105263])

In [13]:
re

     con: array([8.43769499e-14, 1.00070956e-08])
     fun: 1.4421052639885934
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([5.08421052e+00, 2.61857060e-08, 5.21052638e-01])
  status: 0
 success: True
       x: array([2.38827381, 1.51973563, 2.05951041, 1.48301065, 2.27236721,
       1.63845778])

# Problem 2

In [94]:
# A = np.array([[1,4,5,-1,-4,-5],[3,2,9,-3,-2,-9],[-1,3,3,1,-3,-3]])
# b = np.array([0,5,3])
# G = np.array([[1,2,1,-1,-2,-1],[8,-5,3,-8,5,-3]])
# h = np.array([4,7])
A = np.array([[1,4,5],[3,2,9],[-1,3,3]])
b = np.array([0,5,3])
G = np.array([[1,2,1],[8,-5,3]])
h = np.array([4,7])

In [95]:
# Construct the problem.
x = cp.Variable(3)
objective = cp.Minimize(cp.sum_squares(A@x - b))
constraints = [G@x==h]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

In [96]:
# The optimal value for x is stored in `x.value`.x
x_opt = x.value
print(x_opt)
print((A@x_opt-b) @ (A@x_opt-b))
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
for cons in constraints:
    new_h = np.array(cons.dual_value)
    print(new_h)
    
    x = cp.Variable(3)
    objective = cp.Minimize(cp.sum_squares(A@x - b))
    constraints = [G@x==new_h]
    prob = cp.Problem(objective, constraints)

    # The optimal objective value is returned by `prob.solve()`.
    result = prob.solve()
    x_opt = x.value
    # print
    print(x.value, (A@x_opt-b) @ (A@x_opt-b))

[ 1.88689591  1.31222542 -0.51134674]
28.87381495061605
[-7.75218842  0.19092921]
[-3.28139953 -3.62313565  2.77548241] 28.643181337190235


# Problem 3

In [118]:
# Construct the problem.
x = cp.Variable(3)
A = np.array([[1,0,0],[0,1,0],[0,0,1],[-1,-1,0],[-1,0,-1]])
b = np.array(([0,0,0,-1,-1]))
objective = cp.Maximize(cp.sum_squares(x))
constraints = [A@x>=b]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
print(result)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to maximize a function that is convex.

# Problem 7

In [120]:
# Construct the problem.
x0 = cp.Variable(1)
x1 = cp.Variable(1)
objective = cp.Maximize(x0*x1 + x1)
constraints = [x0+x1<=1, x0+x1>=0, x0>=0, x1>=0, x0<=1, x1<=1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
print(result)

/Users/dajinhan/opt/anaconda3/envs/web/lib/python3.9/site-packages/cvxpy/expressions/expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 17 times so far.

  warnings.warn(msg, UserWarning)


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
var993 @ var994

In [116]:
result

0.0

In [85]:
class Optimizer:
    def __init__(self, matrix):
        self.A = matrix
        self.p_r = np.ones(self.A.shape[0]) / self.A.shape[0] 
        self.p_c = np.ones(self.A.shape[1]) / self.A.shape[1]
    
    def optimize_r(self):
        A_eq = np.ones((1, self.p_r.size))
        b_eq = np.ones(1)
        A_ub = self.A
        b_ub = np.zeros(self.p_c.size)

        # Set input parameters for linprog function
        c = np.zeros((1, self.p_r.size + 1))
        c[0, -1] = -1
        A_eq = np.concatenate((A_eq, np.zeros((1,1))), axis=1)
        b_eq = b_eq
        A_ub = np.concatenate((-A_ub, np.ones((1, self.p_c.size))), axis=0).T
        b_ub = b_ub
        
        #print(c.shape, A_eq.shape, b_eq.shape, A_ub.shape, b_ub.shape)
        lp_result = linprog(c=c,
                               A_eq=A_eq,
                               b_eq=b_eq,
                               A_ub=A_ub,
                               b_ub=b_ub,
                               bounds=[(0,1)]*self.p_r.size + [(np.min(self.A), np.max(self.A))])
        return lp_result

    def optimize_c(self):
        A_eq = np.ones((1, self.p_c.size))
        b_eq = np.ones(1)
        A_ub = self.A.T
        b_ub = np.zeros(self.p_r.size)

        # Set input parameters for linprog function
        c = np.zeros((1, self.p_c.size + 1))
        c[0, -1] = -1
        A_eq = np.concatenate((A_eq, np.zeros((1,1))), axis=1)
        b_eq = b_eq
        A_ub = np.concatenate((-A_ub, np.ones((1, self.p_r.size))), axis=0).T
        b_ub = b_ub
        
        #print(c.shape, A_eq.shape, b_eq.shape, A_ub.shape, b_ub.shape)
        lp_result = linprog(c=c,
                               A_eq=A_eq,
                               b_eq=b_eq,
                               A_ub=A_ub,
                               b_ub=b_ub,
                               bounds=[(0,1)]*self.p_c.size + [(np.min(self.A), np.max(self.A))])
        return lp_result
    
    def optimize(self):
        result_r = self.optimize_r().x
        result_c = self.optimize_c().x
        print("probability of row is ", result_r[:-1])
        print("value is ", result_r[-1])
        print("probability of col is ", result_c[:-1])
        print("value is ", result_c[-1])

    
    def print_saddle_point(self):
        mins = np.amin(self.A, axis=1)
        maxs = np.amax(self.A, axis=0)
        max_min = np.max(mins)
        min_max = np.min(maxs)

        if max_min != min_max :
            print("No Saddle Point")
        else :
            row_idxs = np.argwhere(self.A == max_min)
            for row_idx in row_idxs:
                print("(%f, %f)" %(row_idx[0]+1, row_idx[1]+1))
        

# Problem 5

In [9]:
matrix_0 = np.array([[0,5,-2],
                     [-3,0,4],
                     [6,-4,0]])
problem_0 = Optimizer(matrix_0)

problem_0.optimize()
problem_0.print_saddle_point()

probability of row is  [0.36363636 0.34965035 0.28671329]
value is  0.6713286713212918
probability of col is  [0.30769231 0.29370629 0.3986014 ]
value is  0.6713286713185154
No Saddle Point
